In [106]:
#!cat bibliotecas.txt | xargs -n 1 pip3 install

In [107]:
#Importação de bibliotecas
import RPi.GPIO as GPIO
import time, copy
import board, busio, adafruit_vl53l0x
import serial
import numpy as np
from numpy import save

In [108]:
#Setup das portas logicas do Raspibery PI
class SetupGPIO:
    def __init__(self):
        self.objGPIO = GPIO
        self.objGPIO.cleanup() # limpa todos os estados de todas as portas
        self.objGPIO.setmode(GPIO.BCM) #Definindi uso dos numeros das portas por canais
    
    def get_gpio(self):
        return self.objGPIO

In [109]:
class MotorCarro:
    def __init__(self, raspGPIO):
        #GPIO Rodas dianteiras
        self.RDEF = 20
        self.RDDF = 5
        self.RDER = 21
        self.RDDR = 27

        #GPIO Rodas dianteiras
        self.RTEF = 12
        self.RTDF = 22
        self.RTER = 16
        self.RTDR = 17
        self.raspGPIO = raspGPIO
        self.raspGPIO.setup(self.RDEF, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RDDF, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RDER, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RDDR, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RTEF, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RTDF, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RTER, self.raspGPIO.OUT)
        self.raspGPIO.setup(self.RTDR, self.raspGPIO.OUT)
        self.stop()

    def stop(self):
        self.raspGPIO.output(self.RDEF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RDDF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RDER, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RDDR, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTEF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTDF, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTER, self.raspGPIO.LOW)
        self.raspGPIO.output(self.RTDR, self.raspGPIO.LOW)

    def forward(self):
        self.raspGPIO.output(self.RDEF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RDDF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTEF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTDF, self.raspGPIO.HIGH)
    

    def backward(self):
        self.raspGPIO.output(self.RDER, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RDDR, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTER, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTDR, self.raspGPIO.HIGH)
        
    def left_forward(self):
        self.raspGPIO.output(self.RDEF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTEF, self.raspGPIO.HIGH)


    def right_forward(self):
        self.raspGPIO.output(self.RDDF, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTDF, self.raspGPIO.HIGH)

    def left_backward(self):
        self.raspGPIO.output(self.RDER, self.raspGPIO.HIGH)
        self.raspGPIO.output(self.RTER, self.raspGPIO.HIGH)

    def right_backward(self):
        self.raspGPIO.output(self.RDDR, self.raspGPIO.HIGH)
    
    def movimentacarro(self, movimento):
        if movimento==0:
            self.forward()
        elif movimento==1:
             self.backward()
        elif movimento==2:
             self.left_forward()
        elif movimento==3:
             self.right_forward()
        elif movimento==4:
             self.left_backward()
        elif movimento==5:
             self.right_backward()

        time.sleep(5)
        self.stop()


In [110]:
class Gsm_GPS:
    def __init__(self):
        self.port=serial.Serial("/dev/tty8",baudrate=11500, timeout=5  )
        self.port.flush()
        self.habilita_gsm_gps()
    
    def habilita_gsm_gps(self):
        self.port.write('AT\r'.encode())
        self.response()
        self.port.write('AT+CSQ\r'.encode())
        self.response()
        port.write('AT+CPIN?\r')
        response()
    
    def response(self):
        responseCount = 0
        while True:
            resp = self.port.readline().decode()
            print(resp)
            print(responseCount)
            if "OK" in resp:
                break
            if "ERROR" in resp:
                break
            if responseCount > 30:
                break
            else:
                responseCount +=1

In [111]:
#gsmgps = Gsm_GPS()

In [112]:
#movimentação do servo motor

class StepMotor:
    def __init__(self, raspGPIO, radar):
        self.controlPinEsquerda = [26,19,13,6]
        self.controlPinDireita = [6,13,19,26]
        self.posicoesMedicaRadar = [0,50, 100, 150, 200, 250, 299]
        self.medicaoRadar=[]
        self.raspGPIO = raspGPIO
        self.posicao_final = 0 #Esquerda
        
        self.seq1 = [[1,0,0,0], 
               [1,1,0,0],
               [0,1,0,0],
               [0,1,1,0],
               [0,0,1,0],
               [0,0,1,1],
               [0,0,0,1],
               [1,0,0,1]]
        self.seq2 = [
                [1,0,0,1],
                [1,0,0,0], 
                [1,1,0,0],
                [0,1,0,0],
                [0,1,1,0],
                [0,0,1,0],
                [0,0,1,1],
                [0,0,0,1]]
        self.parar_motor()

    def parar_motor(self):
        for pin in self.controlPinEsquerda:
            raspGPIO.setup(pin, raspGPIO.OUT)
            raspGPIO.output(pin, False)    
    
    def moverMotorEsquerda(self):
        self.medicaoRadar=[]
        for i in range(0, 300,1):
            if i in self.posicoesMedicaRadar:
                self.medicaoRadar.append(radar.get_radar_posicao())
                        
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinEsquerda[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)

    def moverMotorDireita(self):
        self.medicaoRadar=[]
        for i in range(0,300,1):
            if i in self.posicoesMedicaRadar:
                self.medicaoRadar.append(radar.get_radar_posicao())
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinDireita[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
    

    def moverMotorDireita_ajuste(self, valor):
        for i in range(0,valor,1):
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinDireita[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
       
    def moverMotorEsquerda_ajuste(self, valor):
        for i in range(0, valor, 1):
            for halfstep in range(8):
                for pin in range(4):
                    self.raspGPIO.output(self.controlPinEsquerda[pin], self.seq1[halfstep][pin])
                    time.sleep(0.001)
        
    def get_posicoes_radar(self):
        if self.posicao_final == 0:
            self.moverMotorEsquerda()
            self.posicao_final = 1
        else:
            self.moverMotorDireita()
            self.posicao_final = 0
            self.medicaoRadar = self.medicaoRadar[::-1] 
            
        return self.medicaoRadar

In [113]:
class Vl53_Radar:
    def __init__(self):
        self.i2c = busio.I2C(board.SCL, board.SDA)
        self.vl53 = adafruit_vl53l0x.VL53L0X(self.i2c)
        self.yarr = []
        self.count = 0

    
    def get_radar(self):
        return self.vl53
    
    def get_radar_posicao(self):
        return int(self.vl53.range / 10)

In [114]:
class CarEnv:
    def __init__(self, motorCarro, radarPosicoes, StepMotor):
        self.actions_space = ['forward', 'backward', 'leftforward', 'rightforward', 'leftbackward', 'rightbackward']
        self.observation_space = np.array(np.zeros([len(radarPosicoes), len(self.actions_space)]))
        self.state = radarPosicoes
        self.done = False
        self.input_size = len(radarPosicoes)
        self.output_size = len(self.actions_space)
        self.motorCar =  motorCarro

    def step(self, action):
        self.take_action(action)
        stepP = copy.deepcopy(self.getReward()), copy.deepcopy(self.getState()), self.done
        return stepP

    def getState(self):
        try:
            self.state = stepMotor.get_posicoes_radar()
        finally:
           stepMotor.parar_motor()
        return self.state
        
    def take_action(self, action):
        movPosition = np.where(action == np.max(action)) 
        l = list(action).index(np.max(action))
        print('action', l, np.max(action), action)
        self.motorCar.movimentacarro(l)

    def getReward(self):
        f = lambda x: 10 if  x>100 else -10
        return f(max(self.state[2:5]))


### Inicialização dos Hiperparametros
Neste parte implementaremos a seguinte parte do código

<img src="imagens/ars_part1.png" width=600 heigth=400>

Inicialmente vamos usar steps_size=10 e epsodes=10 apenas para fins de comparação de resultados, ja que o ambiente é real de um carro autonomo

self.directions = Total de matrizes de pertubações a serem contruidas ***number of directions sampled per iteration N***

self.best_directions = Total de matrizes com recompensas melhores 

Nunca as matrizes de pertubação pode ser maior que as matrizes de recompensas, por isso o uso do assert

***(number of top-performing directions to use b (b < N is allowed only for V1-t and V2-t)***

self.noise = noise ν

In [115]:
class Hiperparametros():
    def __init__(self):
        self.steps = 10
        self.epsodes = 10
        self.lr = 0.02
        self.directions = 6
        self.best_directions = 6
        assert self.best_directions <= self.directions
        self.noise = 0.03
        self.seed = 1
        self.env_nome = ''


Os bons resultados desse algoritmo se dão fortemente por causa da normalização dos dados

De acordo com documento, a normalização é necessário por:

"A normalização de estados usada por V2 é semelhante ao clareamento de dados usado em tarefas de regressão, e
intuitivamente, garante que as políticas atribuam peso igual aos diferentes componentes dos estados. Para
obter intuição de por que isso pode ajudar, suponha que uma coordenada de estado só tenha valores no intervalo
90, 100 enquanto outro componente de estado assume valores na faixa -1, 1. Então, pequenas mudanças em
o ganho de controle em relação à primeira coordenada de estado levaria a mudanças maiores nas ações
então, o mesmo tamanho muda em relação ao segundo componente de estado. Portanto, o clareamento permite
a exploração isotrópica de pesquisa aleatória para ter igual influência sobre os vários componentes de estado"

Neste parte implementaremos a seguinte parte do código

<img src="imagens/ars_part2.png" width=600 heigth=400>


Nesse caso vamos normalizar os valores recebidos pelo radar que varia entre 1 e 899, que seriam equivalentes há 1cm e 89,9cm. 


In [116]:
#normalizacao dos estados (Standardization)
class Normalizacao():
    def __init__(self, inputs):
        self.n = np.zeros(inputs) #agregador de estados descobertos desde o inicio
        self.mean = np.zeros(inputs) #média de todos os valores de input
        self.mean_diff = np.zeros(inputs) #usado para o calculo da variancia
        self.var = np.zeros(inputs) #guarda os valores de variancia
    
    def observe(self, x): 
        # x - array : estados das 6 posições lidas pelo radar
        self.n +=1.
        last_mean = self.mean.copy()
        self.mean += (x - self.mean) / self.n
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean_diff/self.n).clip(min = 1e-2) #minino 0.01

    def normalize(self, inputs):
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        return (inputs - obs_mean) / obs_std

In [117]:
class Politicas():
    def __init__(self, input_size, output_size):
        self.theta = np.zeros((output_size, input_size))
    
    def evaluate(self, input, delta=None, direction=None):
        if direction is None:
            return self.theta.dot(input)
        elif direction == 'positive':
            return (self.theta + hp.noise * delta).dot(input)
        else: 
            return (self.theta - hp.noise * delta).dot(input)
    
    def samples_deltas(self):
        return [np.random.randn(*self.theta.shape) for _ in range(hp.directions)]
    
    def update(self, rollouts, sigma_r):
        step = np.zeros(self.theta.shape)
        for r_pos, r_neg, d in rollouts:
            step == (r_pos - r_neg) * d
        self.theta += hp.learning_rate / (hp.best_directions * sigma_r) * step

In [118]:
def explore(env, normalizer, policy, direction = None, delta=None):
    print("Entra em explorer")
    state = env.getState() #ler o ambiente radar
    print("State", state)

    done = False
    num_plays = 0.
    sum_rewards = 0
    while not done and num_plays < hp.epsodes:
       print(num_plays)
       normalizer.observe(state)
       state = normalizer.normalize(state)
       action = policy.evaluate(state, delta, direction)
       reward, state, done = env.step(action) #aquifazer o agente se mover
       reward = max(min(reward, 1), -1)#evita outlier nas recompensas
       sum_rewards += reward
       num_plays =+1
    return sum_rewards


In [119]:
#Treinando
def train(env, policy, normalizer, hp):
    for step in range(hp.steps):
        print(step)
        #Inicializacao das pertubacoes (deltas) e as recompensas negativas e positivas)
        deltas = policy.samples_deltas()
        ##print('deltas', deltas)
        positive_rewards = [0] * hp.directions
        negative_rewards = [0] * hp.directions

        #obtendo as direcao positiva
        for k in range(hp.directions):
            positive_rewards[k] = explore(env, normalizer, policy, direction='positive', delta=deltas[k])
            print(positive_rewards[k])
        #obtendo recompensa na direcao negativa
        for k in range(hp.directions):
            positive_rewards[k] = explore(env, normalizer, policy,  direction='negative', delta=deltas[k])
        
        #obtendo todas as recompensas positivas e negativas para computar o desvio dessas recompensas
        all_reward = np.array(positive_rewards + negative_rewards)
        sigma_r = all_rewards.std()

        #ordenacao dos rollouts e selecao das melhores direcoes
        scores = {k: max(r_pos, r_neg) for k, (r_pos, r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
        order = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)[:hp.best_directions]
        rollouts = [(positive_rewards[k], negative_rewards[k], deltas [k]) for k in order]
        #atualizacao de politica
        policy.update(rollouts, sigma_r)

        #impressao da recompensa
        reward_evaluation = explore(env, normalizer, policy)
        savez_compressed('positive_rewards.npz', positive_rewards)
        savez_compressed('negative_rewards.npz', negative_rewards)
        print('Step', step, 'Reward:',reward_evaluation )
             

In [120]:

raspGPIO  = SetupGPIO().get_gpio()
radar     = Vl53_Radar()
stepMotor = StepMotor(raspGPIO, radar)
motorCar = MotorCarro(raspGPIO)
carEnv = CarEnv(motorCar, stepMotor.get_posicoes_radar(),stepMotor)


In [121]:
 hp = Hiperparametros()
 np.random.seed(hp.seed)
 police = Politicas(carEnv.input_size, carEnv.output_size)
 normalizer = Normalizacao(carEnv.input_size)
 try:
    train(carEnv,police,normalizer,hp)
 finally:
     stepMotor.parar_motor()
 #Fim

0
Entra em explorer
State [30, 6, 3, 4, 8, 91, 819]
0.0
action 0 0.0 [0. 0. 0. 0. 0. 0.]
1
action 4 0.07378462005637447 [ 0.0569203   0.03718467 -0.02541185  0.02422059  0.07378462 -0.09981963]
1
action 5 0.12437211738008236 [-0.12092791 -0.00285693  0.00485461 -0.03400253 -0.0050712   0.12437212]
1
action 3 0.13523523599577225 [-0.03132879 -0.02036323 -0.0729608   0.13523524 -0.02641863  0.11026353]
1
action 3 0.143913240353949 [-0.00192744 -0.03445581 -0.0074237   0.14391324 -0.02963277  0.07111972]
1
action 3 0.10113411748688161 [-0.10411904 -0.0510647   0.02810385  0.10113412 -0.06832019  0.06025521]
1
action 3 0.07943168880821191 [ 0.04427646  0.07803137 -0.0803608   0.07943169  0.05764328  0.04329572]
1
action 3 0.08699936881296322 [ 0.07368698 -0.03720007 -0.00299896  0.08699937 -0.0074706   0.06416811]
1
action 3 0.07806813355913764 [-0.05659376 -0.04868104  0.0180789   0.07806813 -0.04459651  0.04071203]
1
action 0 0.22043739110316105 [ 0.22043739  0.00781367 -0.05691783 -0.09

KeyboardInterrupt: 

In [88]:
#stepMotor.moverMotorDireita_ajuste(300)
##stepMotor.get_posicoes_radar()
#stepMotor.parar_motor()



In [89]:
motorCar.stop()

SyntaxError: invalid syntax (<ipython-input-70-52dc29c6087b>, line 1)